<a href="https://colab.research.google.com/github/iamBehzad/DES-DRL/blob/main/DES_DRL_14020425.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
pip install DESlib;

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import multiprocessing
import pandas as pd
import pickle
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier


from deslib.dcs import LCA
from deslib.dcs import MLA
from deslib.dcs import OLA
from deslib.dcs import MCB
from deslib.dcs import Rank
from deslib.des import KNORAE, KNORAU, KNOP, METADES, DESKNN, DESClustering
from deslib.static.oracle import Oracle
from deslib.static.single_best import SingleBest
from deslib.util.datasets import make_P2

import sklearn.preprocessing as preprocessing
import scipy.io as sio
import time
import os
import warnings
import math

warnings.filterwarnings("ignore")

#DES-DRL

In [15]:
from sklearn.metrics import accuracy_score
import random
from scipy.stats import mode
from sklearn.neighbors import KNeighborsClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import itertools

#Enviorment

In [16]:
class EnsembleSelectionEnv:
    def __init__(self, X_train, y_train, X_test, pool_classifiers, n_estimators):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test

        self.pool_classifiers=pool_classifiers

        self.max_steps = 20
        self.min_steps = 10
        self.min_accuracy = 0.8

        self.num_classes = len(np.unique(self.y_train))
        self.n_estimators = n_estimators
        self.acc_threshold = 0.6
        self.current_ensemble = []
        self.current_step = 0
        self.current_index = -1
        self.current_state = None
        self.done = False
        self.current_ensemble_num_member = 0

        self.current_sample_X = self.X_test[self.current_index]
        #print('\nself.current_sample_X: ', self.current_sample_X)

        self.k=7
        self.knn_classifier = KNeighborsClassifier(n_neighbors=self.k)
        self.knn_classifier.fit(self.X_train, self.y_train)
        knn_distance, self.knn_indices = self.knn_classifier.kneighbors(self.current_sample_X.reshape(1, -1))
        self.knn = np.array([self.X_train[i] for i in self.knn_indices])[0]
        self.knn_class = np.array([self.y_train[i] for i in self.knn_indices])[0]

        self.classifiers_neighbors_hard_classification = np.zeros((self.n_estimators,len(self.knn)))
        self.classifiers_Overall_accuracy=np.zeros(len(self.pool_classifiers))
        self.classifiers_predictions = np.zeros((self.n_estimators,len(self.knn)))

        for j, c in enumerate(self.pool_classifiers):
            for i, x in enumerate(self.knn):
                self.classifiers_predictions[j][i]= c.predict(x.reshape(1, -1))[0]
                if  self.classifiers_predictions[j][i] == self.knn_class[i]:
                  self.classifiers_neighbors_hard_classification[j][i] = 1
            num_correct = sum(self.classifiers_predictions[j] == self.knn_class)
            self.classifiers_Overall_accuracy[j] = num_correct / len(self.classifiers_predictions[j])

        num_ones = np.random.randint(5, 10)
        lst = [0 for _ in range(self.n_estimators)]
        ones_indices = random.sample(range(self.n_estimators), num_ones)
        self.current_ensemble = np.array([1 if i in ones_indices else 0 for i in range(self.n_estimators)])

        self.current_classifiers = [self.pool_classifiers[i] for i in range(len(self.current_ensemble)) if self.current_ensemble[i] == 1]

        self.ensemble_predictions = np.zeros(len(self.knn))
        self.ensemble_neighbors_hard_classification = np.zeros((len(self.knn)))

        self.ensemble_Overall_accuracy = 0

        self.observation_space = {
            'current_ensemble': (self.current_ensemble,),
            'current_ensemble_num_member': 1,
            'ensemble_predictions': self.ensemble_predictions,
            'ensemble_neighbors_hard_classification': self.ensemble_neighbors_hard_classification,
            'ensemble_Overall_accuracy': 1,
            'classifiers_predictions': self.classifiers_predictions.flatten(),
            'classifiers_neighbors_hard_classification': self.classifiers_neighbors_hard_classification.flatten(),
            'classifiers_Overall_accuracy': self.classifiers_Overall_accuracy,
            'k_nearest_neighbors_class' : self.knn_class,
        }

        self.action_space_n = 3
        self.observation_space = len(*self.observation_space["current_ensemble"]) + \
                                     self.observation_space["current_ensemble_num_member"] + \
                                     len(self.observation_space["ensemble_predictions"]) + \
                                     len(self.observation_space["ensemble_neighbors_hard_classification"]) + \
                                     self.observation_space["ensemble_Overall_accuracy"] + \
                                     len(self.observation_space["classifiers_predictions"]) + \
                                     len(self.observation_space["classifiers_neighbors_hard_classification"]) + \
                                     len(self.observation_space["classifiers_Overall_accuracy"]) + \
                                     len(self.observation_space["k_nearest_neighbors_class"])
        #print("self.observation_space: " , self.observation_space)

    def _update_ensemble_stats(self):
      current_ensemble_num_estimator = np.count_nonzero(self.current_ensemble == 1)
      self.current_ensemble_num_member = current_ensemble_num_estimator / self.n_estimators # Normalized No. Members of Current Ensemble

      for i, x in enumerate(self.knn):
        current_ensemble_classifiers_predictions = [self.classifiers_predictions[j][i] for j in range(len(self.current_ensemble)) if self.current_ensemble[j] == 1]
        mode_current_ensemble_classifiers_predictions = mode(current_ensemble_classifiers_predictions, axis=0, keepdims=True)
        mode_current_ensemble_classifiers_predictions_index = int(mode_current_ensemble_classifiers_predictions.mode[0])
        self.ensemble_predictions[i] = (self.current_classifiers[0].classes_[mode_current_ensemble_classifiers_predictions_index])
        if self.ensemble_predictions[i] == self.knn_class[i]:
          self.ensemble_neighbors_hard_classification[i] = 1
      self.ensemble_Overall_accuracy = sum(self.ensemble_predictions == self.knn_class) / len(self.knn_class)

    def _add_member(self):
      # Add a new member to the ensemble by selecting a pre-trained classifier with accuracy greater than the threshold
      inverted_current_ensemble = [not m for m in self.current_ensemble]
      high_acc_cls = [x > self.acc_threshold for x in self.classifiers_Overall_accuracy]
      eligible_clf = [a and b for a, b in zip(inverted_current_ensemble, high_acc_cls)]
      if all(x == 0 for x in eligible_clf):
          alternative_action = np.random.randint(0, env.action_space_n)
          self.step(alternative_action)
      else:
          chosen_list = [i for i, x in enumerate(eligible_clf) if x != 0]
          best_idx = random.choice(chosen_list)
          new_member = self.pool_classifiers[best_idx]
          self.current_ensemble[best_idx] = 1
          self.current_classifiers.append(new_member)
          self._update_ensemble_stats()

    def _replace_member(self):
      # Replace a member in the ensemble with a new member by selecting a pre-trained classifier with accuracy greater than the threshold
      inverted_current_ensemble = [not m for m in self.current_ensemble]
      high_acc_cls = [x > self.acc_threshold for x in self.classifiers_Overall_accuracy]
      eligible_clf = [a and b for a, b in zip(inverted_current_ensemble, high_acc_cls)]
      if all(x == 0 for x in eligible_clf):
          alternative_action = np.random.randint(0, env.action_space_n)
          self.step(alternative_action)
      elif not all(x == 0 for x in self.current_ensemble):
          chosen_list = [i for i, x in enumerate(eligible_clf) if x != 0]
          best_idx = random.choice(chosen_list)

          replace_idx = np.random.choice(np.where(self.current_ensemble == 1)[0])
          self.current_ensemble[replace_idx] = 0
          self.current_ensemble[best_idx] = 1
          self.current_classifiers.clear()
          self.current_classifiers = [self.pool_classifiers[i] for i in range(len(self.current_ensemble)) if self.current_ensemble[i] == 1]
          self._update_ensemble_stats()

    def _remove_member(self):
        chosen_list = [i for i, x in enumerate(self.current_ensemble) if x != 0]
        if len(chosen_list) > 1 :
          remove_idx = random.choice(chosen_list)
          self.current_ensemble[remove_idx] = 0
          self.current_classifiers.clear()
          self.current_classifiers = [self.pool_classifiers[i] for i in range(len(self.current_ensemble)) if self.current_ensemble[i] == 1]
          self._update_ensemble_stats()
        else :
          alternative_action = np.random.randint(0, env.action_space_n)
          self.step(alternative_action)

    def step(self, action):
      self.current_step += 1
      if self.current_step < self.max_steps:
        if action == 0:
            self._add_member()
        elif action == 1:
            self._replace_member()
        elif action == 2:
            self._remove_member()
        else:
            raise ValueError(f"Invalid action: {action}")

      next_state = self._get_obs()
      self.current_state = next_state

      reward =  self.ensemble_Overall_accuracy

      if self.current_step >= self.max_steps:
          self.done = True
      if self.current_step >= self.min_steps and self.ensemble_Overall_accuracy > self.min_accuracy:
          self.done = True
      if self.done:
          #print('self.current_ensemble_num_member: ', int(self.current_ensemble_num_member*self.n_estimators))
          #print("current_step: ", self.current_step, "next_state: ", next_state, "reward: ", reward, "done: ", self.done)
          #print('self.ensemble_Overall_accuracy: ', self.ensemble_Overall_accuracy)
          return next_state, reward, self.done,{tuple((tuple(self.current_sample_X), tuple(self.current_ensemble), self.current_ensemble_num_member)) }

      return next_state, reward, self.done, {()}

    def reset(self):
        self.current_step = 0
        self.done = False
        self.current_index +=1
        #print('self.X_train.shape', self.X_train.shape)
        #print('self.X_test.shape', self.X_test.shape)
        #print("self.current_index", self.current_index)

        self.current_sample_X = self.X_test[self.current_index]

        knn_distance, self.knn_indices = self.knn_classifier.kneighbors(self.current_sample_X.reshape(1, -1))
        self.knn = np.array([self.X_train[i] for i in self.knn_indices])[0]
        self.knn_class = np.array([self.y_train[i] for i in self.knn_indices])[0]

        for j, c in enumerate(self.pool_classifiers):
            for i, x in enumerate(self.knn):
                self.classifiers_predictions[j][i]= c.predict(x.reshape(1, -1))[0]
                if  self.classifiers_predictions[j][i] == self.knn_class[i]:
                  self.classifiers_neighbors_hard_classification[j][i] = 1
            num_correct = sum(self.classifiers_predictions[j] == self.knn_class)
            self.classifiers_Overall_accuracy[j] = num_correct / len(self.classifiers_predictions[j])

        num_ones = np.random.randint(5, 10)
        lst = [0 for _ in range(self.n_estimators)]
        ones_indices = random.sample(range(self.n_estimators), num_ones)
        self.current_ensemble =np.array( [1 if i in ones_indices else 0 for i in range(self.n_estimators)])

        self.current_state = self._get_obs()
        return self.current_state

    def _get_obs(self):
        obs = {
                'current_ensemble': [*self.current_ensemble] ,
                'current_ensemble_num_member': self.current_ensemble_num_member,
                'ensemble_predictions': self.ensemble_predictions,
                'ensemble_neighbors_hard_classification': self.ensemble_neighbors_hard_classification,
                'ensemble_Overall_accuracy': self.ensemble_Overall_accuracy,
                'classifiers_predictions': np.concatenate(self.classifiers_predictions),
                'classifiers_neighbors_hard_classification': np.concatenate(self.classifiers_neighbors_hard_classification) ,
                'classifiers_Overall_accuracy': self.classifiers_Overall_accuracy ,
                'k_nearest_neighbors_class' : self.knn_class,
              }
        return *obs['current_ensemble'], obs['current_ensemble_num_member'], *obs['ensemble_predictions'],\
               *obs['ensemble_neighbors_hard_classification'],obs['ensemble_Overall_accuracy'],\
               *obs['classifiers_predictions'],\
               *obs['classifiers_neighbors_hard_classification'],*obs['classifiers_Overall_accuracy'],\
               *obs["k_nearest_neighbors_class"]

#Parameter Setting

In [17]:
# if gpu is to be used
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
Tensor = torch.Tensor
LongTensor = torch.LongTensor
seed_value = 23
torch.manual_seed(seed_value)
random.seed(seed_value)
###### PARAMS ######
learning_rate = 0.001
gamma = 0.9999
hidden_layer = 64
replay_mem_size = 50000
batch_size = 32
update_target_frequency = 500
double_dqn = True
report_interval = 10
score_to_solve = 195
clip_error = False
egreedy = 0.9
egreedy_final = 0.01
egreedy_decay = 300

#Deep Q-Learning

In [18]:
class ExperienceReplay(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
    def push(self, state, action, new_state, reward, done):
        transition = (state, action, new_state, reward, done)
        if self.position >= len(self.memory):
            self.memory.append(transition)
        else:
            self.memory[self.position] = transition
        self.position = ( self.position + 1 ) % self.capacity
    def sample(self, batch_size):
        return zip(*random.sample(self.memory, batch_size))
    def __len__(self):
        return len(self.memory)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(number_of_inputs,hidden_layer)
        self.linear2 = nn.Linear(hidden_layer,number_of_outputs)
        #self.activation = nn.Tanh()
        self.activation = nn.ReLU()
    def forward(self, x):
        output1 = self.linear1(x)
        output1 = self.activation(output1)
        output2 = self.linear2(output1)
        return output2

class QNet_Agent(object):
    def __init__(self):
        self.nn = NeuralNetwork().to(device)
        self.target_nn = NeuralNetwork().to(device)
        self.loss_func = nn.MSELoss()
        #self.loss_func = nn.SmoothL1Loss()
        #self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        self.optimizer = optim.RMSprop(params=self.nn.parameters(), lr=learning_rate)
        self.update_target_counter = 0
    def select_action(self,state,epsilon):
        random_for_egreedy = torch.rand(1)[0]
        if random_for_egreedy > epsilon:
            with torch.no_grad():
                state = Tensor(state).to(device)
                action_from_nn = self.nn(state)
                action = torch.max(action_from_nn,0)[1]
                action = action.item()
        else:
            action = np.random.randint(0, env.action_space_n)
        return action
    def optimize(self):
        if (len(memory) < batch_size):
            return
        state, action, new_state, reward, done = memory.sample(batch_size)
        state = Tensor(state).to(device)
        new_state = Tensor(new_state).to(device)
        reward = Tensor(reward).to(device)
        action = LongTensor(action).to(device)
        done = Tensor(done).to(device)
        if double_dqn:
            new_state_indexes = self.nn(new_state).detach()
            max_new_state_indexes = torch.max(new_state_indexes, 1)[1]
            new_state_values = self.target_nn(new_state).detach()
            max_new_state_values = new_state_values.gather(1, max_new_state_indexes.unsqueeze(1)).squeeze(1)
        else:
            new_state_values = self.target_nn(new_state).detach()
            max_new_state_values = torch.max(new_state_values, 1)[0]
        target_value = reward + ( 1 - done ) * gamma * max_new_state_values
        predicted_value = self.nn(state).gather(1, action.unsqueeze(1)).squeeze(1)
        loss = self.loss_func(predicted_value, target_value)
        self.optimizer.zero_grad()
        loss.backward()
        if clip_error:
            for param in self.nn.parameters():
                param.grad.data.clamp_(-1,1)
        self.optimizer.step()
        if self.update_target_counter % update_target_frequency == 0:
            self.target_nn.load_state_dict(self.nn.state_dict())
        self.update_target_counter += 1
        #Q[state, action] = reward + gamma * torch.max(Q[new_state])

def calculate_epsilon(steps_done):
    epsilon = egreedy_final + (egreedy - egreedy_final) * \
              math.exp(-1. * steps_done / egreedy_decay )
    return epsilon

#Train

In [19]:
def Train(num_episodes):
    for i_episode in range(num_episodes):

      state = env.reset()
      step = 0
      frames_total = 0
      done = False
      rewards=[]
      while not done:
          step += 1
          frames_total += 1
          epsilon = calculate_epsilon(frames_total)
          #print("epsilon :", epsilon)
          action = qnet_agent.select_action(state, epsilon)
          new_state, reward, done, info = env.step(action)
          memory.push(state, action, new_state, reward, done)
          qnet_agent.optimize()
          state = new_state
          rewards.append(reward)
          if done:
            break

#Test

In [20]:
def Test(num_episodes):
    selected_ensemble = []
    num_member = []
    classifiers_predictions = []
    overall_accuracy_selected_ensemble = []
    accuracy_selected_ensemble = []
    diversity_selected_ensemble = []
    num_episodes = len(X_test)
    X_test_data = []
    y_hat = []
    total_reward = []

    for i_episode in range(num_episodes):
        state = test_env.reset()
        #step = 0
        frames_total = 0
        done = False
        rewards=0
        while not done:
            #step += 1
            frames_total += 1
            #epsilon = calculate_epsilon(frames_total)
            epsilon = 0
            action = qnet_agent.select_action(state, epsilon)
            new_state, reward, done, info = test_env.step(action)
            memory.push(state, action, new_state, reward, done)
            #qnet_agent.optimize()
            state = new_state
            rewards+=reward
            if done:
              info_list = list(info)
              x=info_list[0][0]
              X_test_data.append(info_list[0][0])
              EoC=info_list[0][1]
              selected_ensemble.append(EoC)
              num_member.append(info_list[0][2])
              current_classifiers = [pool_classifiers[i] for i in range(len(EoC)) if EoC[i] == 1]
              predictions = [c.predict(np.array(x).reshape(1, -1))[0] for c in current_classifiers]
              classifiers_predictions.append(predictions)
              ensemble_predictions = mode(predictions, axis=0, keepdims=True)
              predicted_class_index = ensemble_predictions.mode[0]
              y_hat.append(current_classifiers[0].classes_[predicted_class_index])
              total_reward.append(rewards)
              break
    return y_hat, selected_ensemble, num_member

#Test Platform

In [21]:
# Prepare the DS techniques. Changing k value to 7.

def initialize_ds(pool_classifiers, X_DSEL, y_DSEL, k=7):
    knorau = KNORAU(pool_classifiers, k=k)
    kne = KNORAE(pool_classifiers, k=k)
    desknn = DESKNN(pool_classifiers, k=k)
    ola = OLA(pool_classifiers, k=k)
    lca = LCA(pool_classifiers, k=k)
    mla = MLA(pool_classifiers, k=k)
    mcb = MCB(pool_classifiers, k=k)
    rank = Rank(pool_classifiers, k=k)
    knop = KNOP(pool_classifiers, k=k)
    meta = METADES(pool_classifiers, k=k)

    single_best = SingleBest(pool_classifiers, n_jobs=-1)
    oracle = Oracle(pool_classifiers)
    majority_voting = pool_classifiers

    list_ds = [knorau, kne, desknn, ola, lca, mla, mcb, rank, knop, meta, single_best, oracle]
    methods_names = ['KNORA-U', 'KNORAE', 'DESKNN', 'OLA', 'LCA', 'MLA', 'MCB', 'Rank', 'KNOP', 'META-DES', 'SingleBest', 'Oracle']

    # fit the ds techniques
    for ds in list_ds:
        if ds != majority_voting:
            ds.fit(X_DSEL, y_DSEL)


    return list_ds, methods_names

def save_pool(datasetName,pools):
    path = ExperimentPath + "/Pools/" + datasetName + "_pools.p"
    poolspec = open(path, mode="wb")
    pickle.dump(pools, poolspec)
    poolspec.close()
def load_pool(datasetName):
    path = ExperimentPath + "/Pools/" + datasetName + "_pools.p"
    poolspec = open(path, mode="rb")
    return pickle.load(poolspec)

def save_model(tec_name,datasetName,ds):
    path = ExperimentPath + "/Models/" + tec_name +"_"+datasetName + "_model.p"
    poolspec = open(path, mode="wb")
    pickle.dump(ds, poolspec)
    poolspec.flush()
    poolspec.close()
def load_model(tec_name,datasetName):
    path = ExperimentPath + "/Models/" + tec_name +"_"+ datasetName + "_model.p"
    poolspec = open(path, mode="rb")
    return pickle.load(poolspec)

def save_results(tec_name,datasetName,accuracy,labels,yhat):
    path = ExperimentPath + "/Results/" + tec_name +"_"+datasetName + "_result.p"
    poolspec = open(path, mode="wb")
    pickle.dump(accuracy, poolspec)
    pickle.dump(labels, poolspec)
    pickle.dump(yhat, poolspec)
    poolspec.flush()
    poolspec.close()

def pool_generator(datasetName):
    state = 0
    pools = []
    for itr in range(0, no_itr):
        rng = np.random.RandomState(state)
        [X_train, X_test, X_DSEL, y_train, y_test, y_DSEL] =  np.load('/content/drive/MyDrive/Experiment1/Datasets/' + datasetName +str(itr)+'.npy',allow_pickle=True)

        learner = Perceptron(max_iter=100, tol=10e-3, alpha=0.001, penalty=None, random_state=rng)
        model = CalibratedClassifierCV(learner, cv=5,method='isotonic')
        pool_classifiers = BaggingClassifier(model, n_estimators=NO_classifiers, bootstrap=True, max_samples=1.0, random_state=rng)
        pool_classifiers.fit(X_train,y_train)
        pools.append(pool_classifiers)

    path = ExperimentPath + "/Pools/" + datasetName + "_pools.p"
    poolspec = open(path, mode="wb")
    pickle.dump(pools, poolspec)

def model_setup(datasetName):
    global methods_names
    pools = load_pool(datasetName)
    ds_matrix = []
    for itr in range(no_itr):
        pool_classifiers = pools[itr]
        [X_train, X_test, X_DSEL, y_train, y_test, y_DSEL] = np.load('/content/drive/MyDrive/Experiment1/Datasets/' + datasetName + str(itr) + '.npy', allow_pickle=True)
        list_ds, methods_names = initialize_ds(pool_classifiers,X_DSEL,y_DSEL)
        ds_matrix.append(list_ds)

    for tec in range(NO_techniques):
        ds_tec = []
        for itr in range(no_itr):
            ds_tec.append(ds_matrix[itr][tec])
        save_model(methods_names[tec],datasetName,ds_tec)

def evaluate_model(datasetName):
    for tec in range(NO_techniques):
        results = []
        labels = []
        yhat = []
        ds_tec = load_model(methods_names[tec],datasetName)
        for itr in range(no_itr):
            [X_train, X_test, X_DSEL, y_train, y_test, y_DSEL] = np.load('/content/drive/MyDrive/Experiment1/Datasets/' + datasetName + str(itr) + '.npy',  allow_pickle=True)
            labels.append(y_test)
            results.append(ds_tec[itr].score(X_test, y_test) * 100)
            if methods_names[tec] == 'Oracle':
                yhat.append(ds_tec[itr].predict(X_test,y_test))
            else:
                yhat.append(ds_tec[itr].predict(X_test))

        save_results(methods_names[tec],datasetName,results,labels,yhat)

def convert_datasets(datasetName):
    redata = sio.loadmat("/content/drive/MyDrive/DES_DataSets2/" + datasetName + ".mat")
    data = redata['dataset']
    X = data[:, 0:-1]
    y = data[:, -1]
    print(datasetName, "is readed.")
    state = 0
    print(datasetName, ': ', X.shape)


    # ### ### ### ### ### ### ### ### ###
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(y)
    X[np.isnan(X)] = 0
    # #### **** #### **** #### **** #### **** #### **** #### ****
    # scaler = preprocessing.MinMaxScaler()
    # X = scaler.fit_transform(X)
    # #### **** #### **** #### **** #### **** #### **** #### ****


    yhat = np.zeros((no_itr, math.ceil(len(y) / 4)))
    for itr in range(0, no_itr):
        # rand = np.random.randint(1,10000,1)
        rng = np.random.RandomState(itr)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y,
                                                            random_state=rng)  # stratify=y
        X_DSEL, X_test, y_DSEL, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test,
                                                          random_state=rng)  # stratify=y_test
        yhat[itr, :] = y_test


        scaler = preprocessing.MinMaxScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        X_DSEL = scaler.transform(X_DSEL)

        np.save('/content/drive/MyDrive/Experiment1/Datasets/'+ datasetName+ str(itr) ,[X_train,X_test,X_DSEL,y_train,y_test,y_DSEL])

datasets = {
    #     Data set of DGA1033 report
#    "Adult",
#    "Banana",
    "Heart",
    "ILPD",
#   "Vehicle",
#    "Glass",
#    "Pima",
#    "Sonar",
#    "Ecoli"
#    "Wine"
#    "Audit",
#    "Banknote",
#    "Blood",
#    "Breast",
#    "Car",
#    "Datausermodeling",
#    "Faults",
#    "German",
#    "Haberman",
#    "Ionosphere",
#    "Laryngeal1",
#    "Laryngeal3",
#    "Lithuanian",
#    "Liver",
#    "Mammographic",
#    "Monk2",
#    "Phoneme",
#    "Pima",
#    "Sonar",
#    "Statlog",
#    "Steel",
#    "Thyroid",
#    "Vertebral",
#    "Voice3",
#    "Weaning",
}

datasets = sorted(datasets)

ExperimentPath = "/content/drive/MyDrive/Experiment1"
NO_classifiers =100
no_itr = 20
generate_pools = True
do_train = True
do_evaluate = True

methods_names = ['KNORA-U', 'KNORAE', 'DESKNN', 'OLA', 'LCA', 'MLA', 'MCB', 'Rank', 'KNOP', 'META-DES', 'SingleBest', 'Oracle']
NO_techniques = len(methods_names)

start = time.time()

for datasetName in datasets:
    # try:
    print(datasetName)
    convert_datasets(datasetName)
    if generate_pools:
        pool_generator(datasetName)
    if do_train:
        t1 = time.time()
        model_setup(datasetName)
        print("Train time",time.time()-t1)
    if do_evaluate:
        t1 = time.time()
        evaluate_model(datasetName)
        print("Test time", time.time() - t1)

Heart
Heart is readed.
Heart :  (270, 13)
Train time 88.24102520942688
Test time 229.97459268569946
ILPD
ILPD is readed.
ILPD :  (583, 10)
Train time 90.53558492660522
Test time 404.02886986732483


#Run DES-DRL Method

In [22]:
start = time.time()
for datasetName in datasets:
  pools = load_pool(datasetName)
  results = []
  labels = []
  yhat = []
  for itr in range(no_itr):
    pool_classifiers = pools[itr]
    [X_train, X_test, X_DSEL, y_train, y_test, y_DSEL] = np.load('/content/drive/MyDrive/Experiment1/Datasets/' + datasetName + str(itr) + '.npy', allow_pickle=True)
    env = EnsembleSelectionEnv(X_DSEL, y_DSEL, X_DSEL, pool_classifiers, NO_classifiers) # Train
    test_env=EnsembleSelectionEnv(X_DSEL, y_DSEL, X_test, pool_classifiers,NO_classifiers) # Test
    number_of_inputs = env.observation_space
    number_of_outputs = env.action_space_n
    memory = ExperienceReplay(replay_mem_size)
    qnet_agent = QNet_Agent()
    print('iteration: ' , itr, 'th...')
    num_episodes = len(X_DSEL)-1
    t1 = time.time()
    Train(num_episodes)
    print("Train time",time.time()-t1)
    t1 = time.time()
    num_episodes = len(X_test)-1
    y_hat, selected_ensemble, num_member = Test(num_episodes)
    print("Test time", time.time() - t1)
    labels.append(y_test)
    print("Accuracy ", itr, "is : ", accuracy_score( y_test, y_hat) * 100)
    results.append(accuracy_score( y_test, y_hat) * 100)
    yhat.append(y_hat)
  save_results("DES_DRL",datasetName,results,labels,yhat)

iteration:  0 th...
Train time 133.2876741886139
Test time 114.95684289932251
Accuracy  0 is :  82.35294117647058
iteration:  1 th...
Train time 126.85012412071228
Test time 114.39574003219604
Accuracy  1 is :  79.41176470588235
iteration:  2 th...
Train time 127.34954571723938
Test time 114.63319873809814
Accuracy  2 is :  75.0
iteration:  3 th...
Train time 123.765310049057
Test time 117.5309066772461
Accuracy  3 is :  83.82352941176471
iteration:  4 th...
Train time 126.56482172012329
Test time 113.1367073059082
Accuracy  4 is :  85.29411764705883
iteration:  5 th...
Train time 120.53467297554016
Test time 117.16621017456055
Accuracy  5 is :  86.76470588235294
iteration:  6 th...
Train time 131.0599720478058
Test time 116.13318157196045
Accuracy  6 is :  77.94117647058823
iteration:  7 th...
Train time 122.55007600784302
Test time 113.78835463523865
Accuracy  7 is :  85.29411764705883
iteration:  8 th...
Train time 124.69480872154236
Test time 117.60347366333008
Accuracy  8 is :  85

In [27]:
methods_names = ['DES_DRL','KNORA-U', 'KNORAE', 'DESKNN', 'OLA', 'LCA', 'MLA', 'MCB', 'Rank', 'KNOP', 'META-DES', 'SingleBest', 'Oracle']
for d in datasets:
  accuracy_dict = {}
  for m in methods_names:
      accuracy_list = []
      path = ExperimentPath + "/Results/" + m + "_" + d + "_result.p"
      with open(path, mode="rb") as f:
          accuracy = pickle.load(f)
      accuracy_list.append(accuracy)
      accuracy_dict[m] = np.array(accuracy_list).reshape(-1,)
  df = pd.DataFrame(accuracy_dict)
  # Calculate mean, std, and rank of mean for each column
  mean = df.mean()
  std = df.std()
  rank = mean.rank(ascending=False)
  df.loc['mean'] = mean
  df.loc['std'] = std
  df.loc['rank'] = rank
  save_path='/content/drive/MyDrive/Experiment1/Results_xls/'
  writer = pd.ExcelWriter(save_path + d +"_accuracy_results.xlsx")
  df.to_excel(writer)
  writer.save()
  #print(df)